In [1]:
import requests
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
load_dotenv()

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

api_wrapper = WikipediaAPIWrapper(top_k_results=1)

# Define a function to call Wikipedia with limited output
def wikipedia_caller(query: str) -> str:
    """This function queries Wikipedia and returns a condensed summary."""
    full_summary = api_wrapper.run(query)

    # Extract the first few sentences to condense the response
    important_sentences = ". ".join(full_summary.split(". ")[:3]) + "."  # First 3 sentences

    return important_sentences

# Define input parameters for the Wikipedia tool
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# Describe the tool's purpose
description = (
    "A tool that searches Wikipedia. "
    "Useful for answering general questions about people, places, companies, facts, historical events, or other subjects."
)

# Create a structured tool for Wikipedia search
wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia_search",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)



template = """You are a helpful assistant specialized in searching information about destination locations.
{input}
{agent_scratchpad}"""
prompt = ChatPromptTemplate.from_template(template=template)
tools = [wiki_tool]
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Wikipedia Search Agent for destination location is ready.")

question = input("Enter destination location: ")
response = agent_executor.invoke({"input": question})
print(f"{response}")


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3460: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be us

Wikipedia Search Agent for destination location is ready.


> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `{'query': 'Ski Dubai'}`


Page: Ski Dubai
Summary: Ski Dubai is an indoor ski resort with 22,500 square meters of indoor ski area. The park maintains a temperature of −1 to 2 °C (30 to 36 °F) throughout the year. It is a part of the Mall of the Emirates, one of the largest shopping malls in the world, located in Dubai, United Arab Emirates.Ski Dubai is an indoor ski resort covering 22,500 square meters. It maintains a temperature of -1 to 2 °C (30 to 36 °F) year-round and is located within the Mall of the Emirates in Dubai, United Arab Emirates.

> Finished chain.
{'input': 'Ski Dubai', 'output': 'Ski Dubai is an indoor ski resort covering 22,500 square meters. It maintains a temperature of -1 to 2 °C (30 to 36 °F) year-round and is located within the Mall of the Emirates in Dubai, United Arab Emirates.'}
